<a href="https://colab.research.google.com/github/kanade9/Implementation-Transformer/blob/master/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! mkdir data

In [0]:
# ここで一括import
import glob, os, io, string, urllib.request, tarfile, re, torchtext, random,zipfile
from torchtext.vocab import Vectors

In [0]:
# 単語ベクトルのダウンロード 5ふんぐらいかかる
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = "./data/wiki-news-300d-1M.vec.zip"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)
zip = zipfile.ZipFile("./data/wiki-news-300d-1M.vec.zip")
zip.extractall("./data/")  # ZIPを解凍
zip.close() 

In [0]:
# Download IMDb dataset
save_path = "./data/aclImdb_v1.tar.gz"
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)
tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/')
tar.close()

In [0]:
path_list = ['./data/aclImdb/train/pos', './data/aclImdb/train/neg', './data/aclImdb/test/pos',
             './data/aclImdb/test/neg']
cnt=0
for dataset_path in path_list:
    if 'train' in dataset_path:
        f = open('./data/IMDb_train.tsv', 'w')
    else:
        f = open('./data/IMDb_test.tsv', 'w')

    # ダウンロードしてきたファイルの整形を行う
    for fname in glob.glob(os.path.join(dataset_path, '*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            text = text.replace('\t', " ")
            text = text + '\t' + '1' + '\t' + '\n'
            f.write(text)
    f.close()

In [0]:
# ここから前処理
def preprocessing_text(text):
    text = re.sub('<br />', '', text)

    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        text = text.replace(p, " ")

    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


# 分かち書き
def tokenizer_punctuation(text):
    return text.strip().split()


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


print(tokenizer_with_preprocessing('I like cats.'))

['I', 'like', 'cats', '.']


In [0]:
# DataLoaderの作成
# init_token 全部の文章で文頭に入れておく単語
# eos_token 全部の文章で文末に入れておく単語

max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True,
                            fix_length=max_length, init_token="<cls>", eos_token="<eos>")

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# Datasetの作成
# 訓練および検証データセットを分ける。
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])
print('訓練および検証のデータの数',len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))

訓練および検証のデータの数 25000
1つ目の訓練および検証のデータ {'Text': ['a', 'compelling', 'thriller', ',', '10', 'december', '2005', 'author', 'littlehammer16787', 'from', 'united', 'statesjust', 'causestarring', 'sean', 'connery', ',', 'laurence', 'fishburne', ',', 'and', 'blair', 'underwood', '.', 'a', 'liberal', ',', 'though', 'good', 'hearted', 'harvard', 'law', 'professor', 'paul', 'armstrong', 'is', 'convoked', 'to', 'the', 'flordia', 'everglades', 'by', 'unjustly', 'convicted', 'black', 'guy', 'bobby', 'earl', '.', 'confessing', 'that', 'sadistic', ',', 'cold', 'hearted', 'cops', 'vilifyied', 'and', 'beat', 'him', 'to', 'a', 'pulp', 'to', 'get', 'the', 'confession', 'of', 'a', 'gruesome', 'murder', 'of', 'an', 'eleven', 'year', 'old', 'girl', '.', 'as', 'he', 'digs', 'further', 'and', 'further', 'into', 'the', 'mysterious', 'case', 'he', 'realizes', 'that', 'bobby', 'earl', 'is', 'a', 'victim', 'of', 'discrimination', '.', 'that', 'the', 'black', 'police', 'detective', 'lt', '.', 'tanny', 'brown', 'of',

In [0]:
# 訓練データのtsvファイルを作成します

f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [0]:
# Datasetの作成
# 訓練および検証データセットを分ける。
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])
print('訓練および検証のデータの数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))

# 訓練データ:検証データを8:2で分ける。
train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))
# これで、訓練、検証、テスト3つのDatasetの作成が完了.

# debug
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('１つ目の訓練データ', vars(train_ds[0]))

訓練および検証のデータの数 25000
1つ目の訓練および検証のデータ {'Text': ['a', 'compelling', 'thriller', ',', '10', 'december', '2005', 'author', 'littlehammer16787', 'from', 'united', 'statesjust', 'causestarring', 'sean', 'connery', ',', 'laurence', 'fishburne', ',', 'and', 'blair', 'underwood', '.', 'a', 'liberal', ',', 'though', 'good', 'hearted', 'harvard', 'law', 'professor', 'paul', 'armstrong', 'is', 'convoked', 'to', 'the', 'flordia', 'everglades', 'by', 'unjustly', 'convicted', 'black', 'guy', 'bobby', 'earl', '.', 'confessing', 'that', 'sadistic', ',', 'cold', 'hearted', 'cops', 'vilifyied', 'and', 'beat', 'him', 'to', 'a', 'pulp', 'to', 'get', 'the', 'confession', 'of', 'a', 'gruesome', 'murder', 'of', 'an', 'eleven', 'year', 'old', 'girl', '.', 'as', 'he', 'digs', 'further', 'and', 'further', 'into', 'the', 'mysterious', 'case', 'he', 'realizes', 'that', 'bobby', 'earl', 'is', 'a', 'victim', 'of', 'discrimination', '.', 'that', 'the', 'black', 'police', 'detective', 'lt', '.', 'tanny', 'brown', 'of',

In [0]:
# torchtextで単語ベクトルとして英語学習済みモデルを利用する。
english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')
print("1単語を表現する次元数:", english_fasttext_vectors.dim)
print("単語数:", len(english_fasttext_vectors.itos))

# ベクトル化したバージョンのボキャブラリーを作成する
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルの確認を行う
print(TEXT.vocab.vectors.shape)
print(TEXT.vocab.vectors)
print(TEXT.vocab.stoi)

# DataLoaderの作成
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=24, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=24, train=False, sort=False)

# 検証データセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

1単語を表現する次元数: 300
単語数: 999994
torch.Size([17912, 300])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0452,  0.1699,  0.1673,  ..., -0.1171,  0.1026,  0.0478],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
defaultdict(<function _default_unk_index at 0x7fd32423d620>, {'<unk>': 0, '<pad>': 1, '<cls>': 2, '<eos>': 3, 'the': 4, '.': 5, ',': 6, 'and': 7, 'a': 8, 'of': 9, 'to': 10, 'is': 11, 'it': 12, 'in': 13, 'i': 14, 'this': 15, 'that': 16, 's': 17, 'was': 18, 'as': 19, 'movie': 20, 'with': 21, 'for': 22, 'but': 23, 'film': 24, 't': 25, 'you': 26, 'on': 27, 'not': 28, 'he': 29, 'his': 30, 'are': 31, 'have': 32, 'be': 33, 'one': 34, 'all': 35, 'at': 36, 'they': 37, 'by': 38, 'an': 39, 'who': 40, 'so': 41, 'from':